# Notebook 1: Creating Items for Building Complexes

This notebook implements the first step of creating of the Klosterdatenbank-to-Factgrid-Workflow which is to create Items for the building complexes. It contains describing elements about the underlying data model and the workflow in general, as well as specific instructions in order to run the notebook. Markdown cells containing describing elements are marked as `#description`. Instructional sections are marked as `#instruction`.

Strictly speaking, the monastery database does not contain dedicated information on building complexes. Information on where a religious community had its place of operation is stored in the `gs_monastery_location` table. This table assigns each row of a religious community (`gsn_id`) to a location (`place_id`) and, if known, specific coordinates within this location (`longitude`, `latitude`). Such an assignment implies that the community lived or worked at this location at a certain point in time. At this point, we make the central assumption that a building complex of some kind, consisting of at least one building, must have existed. Accordingly, the building complexes created in this step represent both a row from the `gs_monastery_location` table and thus an assignment of a monastery to a specific location, as well as physical buildings in which religious communities worked and which may have continued to exist before or after their use and have experienced other use scenarios.

## Preparations

The notebook requires the following libraries to run. If an error occurs, make sure the libraries are installed on your system.

In [239]:
import pandas as pd
import numpy as np
import os
import csv

First, the export files are loaded into [Dataframes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). The dataframes are stored in a dictionary with the keys being the filenames, for easier access.

In [240]:
# Load Access exports
from helper_functions import load_files_from_folder, query_factgrid

export_files = load_files_from_folder("data/exports_monasteryDB", "xlsx")

# Create dataframes for each table
dataframes = {key: pd.read_excel(value) for key, value in export_files.items()}

# Add dataframe for monasteries in factGrid (stored in a different directory)
dataframes["building_complexes_in_factgrid"] = query_factgrid("building_complexes")
dataframes["monasteries_in_factgrid"] = query_factgrid("monasteries")

Since `gs_monastery_location` does not contain the name of the monasteries, the table is joined to `gs_monastery` to extract the missing information. The resulting table is cut down to the relevant columns. The resulting dataframe is being filtered to only contain information on religious comunities that have the status "online", meaning they are not currently worked on anymore. Finally, to make sure that no duplicate building complexes are being created, the table is filtered against the existing building complexes in FactGrid.

In [241]:
# Merge gs_monastery_location and gs_monastery
merged_df = pd.merge(dataframes["gs_monastery_location"], dataframes["gs_monastery"], left_on='gsn_id', right_on='id_gsn', how='left')
# Filter for status 'online'
online_df = merged_df[merged_df["status"] == "Online"]
# Define columns to drop
drop_columns = [
    "relocated", 
    "comment", 
    "main_location", 
    "diocese_id", 
    "id_monastery", 
    "date_created", 
    "created_by_user", 
    "patrocinium",
    "selection", 
    "processing_status", 
    "gs_persons", 
    "selection_criteria", 
    "last_change", 
    "changed_by_user", 
    "founder"
]
# Prepare dataframe by dropping unnecessary columns
prepared_df = online_df.drop(drop_columns, axis="columns")
prepared_df = prepared_df[~prepared_df["id_monastery_location"].isin(dataframes["building_complexes_in_factgrid"]["GSVocabTerm"].str.split("Location").str[-1].astype(int))]
print(f"{len(prepared_df[prepared_df["id_monastery_location"].isin(dataframes["building_complexes_in_factgrid"]["GSVocabTerm"].str.split("Location").str[-1].astype(int))])} locations are already in FactGrid and filtered out")
prepared_df

0 locations are already in FactGrid and filtered out


,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,Unnamed: 0_y,id_gsn,status,note,monastery_name
0,26,7956,705,60419,1291,NaN,NaN,1525,NaN,NaN,10.419200,49.503200,Windsheim,2169,60419,Online,NaN,Augustinereremitenkloster Windsheim
1,541,17231,46484701,11787,1285,NaN,1285 erstmals erwähnt,1526,NaN,1526 noch existent,NaN,NaN,Viborg,7030,11787,Online,Lokalisierung nach Ortsmittelpunkt.,"Johanniterkommende Viborg, Dänemark"
2,924,5344,209,30016,1250,1260.0,um 1255,1527,NaN,NaN,9.270021,50.749630,NaN,3442,30016,Online,NaN,"Augustinereremitenkloster zum Erlöser, Alsfeld"
3,982,3064,1420,20004,1357,NaN,NaN,1534,NaN,um 1534,8.836796,48.521692,NaN,2624,20004,Online,NaN,Franziskanerinnenkloster Bondorf
4,1357,68,2803,68,1437,NaN,NaN,1647,NaN,NaN,NaN,NaN,NaN,4227,68,Online,1647 starb die letzte Konventualin. Koordinate...,Augustinerinnenkloster Eldagsen (Marienthal)
5,2440,13102,46483121,8028,1409,NaN,NaN,1415,NaN,NaN,4.894167,52.369722,Amsterdam,5690,8028,Online,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd..."
6,2441,13103,46483121,8028,1415,NaN,NaN,1579,NaN,NaN,4.897115,52.371050,Amsterdam,5690,8028,Online,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd..."
8,3283,5599,29638,30279,1163,1185.0,1163/1185,1568,NaN,NaN,8.140393,50.449848,NaN,1536,30279,Online,1568 wurde das Kloster in eine Erziehungsansta...,Prämonstratenserinnenstift Beselich
9,3819,7133,6305,50211,1245,1295.0,vor 1295,1802,NaN,NaN,6.953623,50.944112,NaN,1440,50211,Online,Der Beginenkonvent Loershaus in der Stolkgasse...,"Franziskanerterziarinnenkloster St. Ignatius, ..."
10,5090,3754,10109,20694,1345,NaN,vor 1345,1493,NaN,NaN,8.934051,48.476825,Rottenburg,1643,20694,Online,NaN,"Franziskanerinnenkloster, gen. Am Sülchen Tor,..."


To double-check potential duplicates, the following cell finds buildings complexes that are connected to monasteries already existent in FactGrid. If the resulting DataFrame is empty, all building complexes will be linked to newly created monastery items.

In [242]:
existing_monasteries = prepared_df[prepared_df["gsn_id"].isin(dataframes["monasteries_in_factgrid"]["KlosterdatenbankID"].astype(int))]
existing_monasteries

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,Unnamed: 0_y,id_gsn,status,note,monastery_name


## Labels

It is expected that items in FactGrid have a label in at least one language. While the FactGrid ID (also referred to as the "Q-Number") uniquely identifies the item, the label serves to capture the name of the item in everyday language. The label is also indexed for text-based search. The naming of the items created in this project follows the following rule:
- For the religious communities, the name from the monastery database is used as the label, for example "Zisterzienserkloster Georgenzell".
- For the building complexes, the labels are constructed according to the following schema: `Gebäudekomplex <monastery_name> [(<location_name>)]`. Here, `monastery_name` is again the name of the religious community from the `gs_monastery` table. `location_name` is a column of the `gs_monastery_location` table. In this column, if available, the specific name given to this location is stored. 

For example, the "Benediktinerinnenkloster Mielen, Sint-Truiden, Belgien" (GSN [11665](https://klosterdatenbank.adw-goe.de/gsn/11665)) has two locations in the Belgian town of Sint-Truiden, namely the location "Sint Truiden" and the location "Metsteren" (see Figure). The constructed labels are then "Gebäudekomplex Benediktinerinnenkloster Mielen, Sint-Truiden, Belgien (Sint-Truiden)" and "Gebäudekomplex Benediktinerinnenkloster Mielen, Sint-Truiden, Belgien (Metsteren)". However, location names are not available in all these cases, which can lead to duplicates in the labels. These are displayed in the workflow, so that location names can be added to distinguish them better.

<img src="documentation-images/Standorte GSN11665.png" alt="Monastery Locations of GSN 11665" width="500">

*Figure 1: Building Complexes of the Benedictine nun's monastery Mielen in Sint-Truiden, Belgium (GSN 11665). Base-Layer: OpenStreetMap.*

The following cell constructs the location names and saves them in a new column called "Lde" (see [Quickstatements specification](https://www.wikidata.org/wiki/Help:QuickStatements#Adding_labels,_aliases,_descriptions_and_sitelinks)).

In [243]:
from helper_functions import construct_description
# 1. Create new column with labels
prepared_df['Lde'] = "Gebäudekomplex " + prepared_df["monastery_name"].str.cat(prepared_df["location_name"].fillna(''), sep=" (") +")"
for index, row in prepared_df.iterrows():
    prepared_df.loc[index, "Dde"] = construct_description(row["location_name"], row["monastery_name"], row["location_begin_taq"], row["location_begin_tpq"], row["location_end_taq"], row["location_end_tpq"])
# 2. If necessary, delete empty brackets at end of labels
prepared_df['Lde'] = prepared_df["Lde"].str.replace(r'\(\)', '', regex=True).apply(lambda x: f'\"{x.strip()}\"')
prepared_df["Dde"] = prepared_df["Dde"].apply(lambda x:f'\"{x}\"')
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,Unnamed: 0_y,id_gsn,status,note,monastery_name,Lde,Dde
0,26,7956,705,60419,1291,NaN,NaN,1525,NaN,NaN,10.419200,49.503200,Windsheim,2169,60419,Online,NaN,Augustinereremitenkloster Windsheim,"""Gebäudekomplex Augustinereremitenkloster Wind...","""Gebäudekomplex Windsheim des Augustinereremit..."
1,541,17231,46484701,11787,1285,NaN,1285 erstmals erwähnt,1526,NaN,1526 noch existent,NaN,NaN,Viborg,7030,11787,Online,Lokalisierung nach Ortsmittelpunkt.,"Johanniterkommende Viborg, Dänemark","""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Gebäudekomplex Viborg der Johanniterkommende ..."
2,924,5344,209,30016,1250,1260.0,um 1255,1527,NaN,NaN,9.270021,50.749630,NaN,3442,30016,Online,NaN,"Augustinereremitenkloster zum Erlöser, Alsfeld","""Gebäudekomplex Augustinereremitenkloster zum ...","""Gebäudekomplex des Augustinereremitenklosters..."
3,982,3064,1420,20004,1357,NaN,NaN,1534,NaN,um 1534,8.836796,48.521692,NaN,2624,20004,Online,NaN,Franziskanerinnenkloster Bondorf,"""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Gebäudekomplex des Franziskanerinnenklosters ..."
4,1357,68,2803,68,1437,NaN,NaN,1647,NaN,NaN,NaN,NaN,NaN,4227,68,Online,1647 starb die letzte Konventualin. Koordinate...,Augustinerinnenkloster Eldagsen (Marienthal),"""Gebäudekomplex Augustinerinnenkloster Eldagse...","""Gebäudekomplex des Augustinerinnenklosters El..."
5,2440,13102,46483121,8028,1409,NaN,NaN,1415,NaN,NaN,4.894167,52.369722,Amsterdam,5690,8028,Online,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert..."
6,2441,13103,46483121,8028,1415,NaN,NaN,1579,NaN,NaN,4.897115,52.371050,Amsterdam,5690,8028,Online,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert..."
8,3283,5599,29638,30279,1163,1185.0,1163/1185,1568,NaN,NaN,8.140393,50.449848,NaN,1536,30279,Online,1568 wurde das Kloster in eine Erziehungsansta...,Prämonstratenserinnenstift Beselich,"""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Gebäudekomplex des Prämonstratenserinnenstift..."
9,3819,7133,6305,50211,1245,1295.0,vor 1295,1802,NaN,NaN,6.953623,50.944112,NaN,1440,50211,Online,Der Beginenkonvent Loershaus in der Stolkgasse...,"Franziskanerterziarinnenkloster St. Ignatius, ...","""Gebäudekomplex Franziskanerterziarinnenkloste...","""Gebäudekomplex des Franziskanerterziarinnenkl..."
10,5090,3754,10109,20694,1345,NaN,vor 1345,1493,NaN,NaN,8.934051,48.476825,Rottenburg,1643,20694,Online,NaN,"Franziskanerinnenkloster, gen. Am Sülchen Tor,...","""Gebäudekomplex Franziskanerinnenkloster, gen....","""Gebäudekomplex Rottenburg des Franziskanerinn..."


As mentioned above, there might be duplicate labels in cases where locations don't have an explicit name. Since they still can be distinguished from another by their identifier and coordinates, this is not necessarily a problem. However, the following cell will create a list of all the duplicate labels so that they can be examined.

**In order to resolve the duplicates**

1. Open and inspect the table located at `data/intermediate_results/duplicate_building_complex_labels.xslx`
2. Add location names in the monastery database
3. Create new exports from the monastery database and replace `data/exports_monasteryDB/gs_monastery.xlsx` and `data/exports_monasteryDB/gs_monastery_location.xlsx` with the new files
4. Re-run the notebook. The cell below now should no longer contain the duplicates you resolved. 

In [244]:
duplicated_building_complex_labels = prepared_df[prepared_df.duplicated(subset="Lde", keep=False)]
duplicated_building_complex_labels.to_excel('data/intermediate_results/duplicate_building_complex_labels.xlsx')
duplicated_building_complex_labels

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,Unnamed: 0_y,id_gsn,status,note,monastery_name,Lde,Dde
5,2440,13102,46483121,8028,1409,NaN,NaN,1415,NaN,NaN,4.894167,52.369722,Amsterdam,5690,8028,Online,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert..."
6,2441,13103,46483121,8028,1415,NaN,NaN,1579,NaN,NaN,4.897115,52.371050,Amsterdam,5690,8028,Online,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert..."


### Translation of Labels

FactGrid is a multilingual platform. Therefore, the labels for the monasteries and building complexes should not only be created in German, but also in English. Due to the heterogeneity of the monastery names in the database, a rule-based translation is difficult to implement. Instead, a Large-Language Model was used. The model, prompting, and details of the translation are described in more detail in the notebook "1a - Translation". We are using the [GWDG/KISSKI API](https://docs.hpc.gwdg.de/services/chat-ai/index.html), so in order to execute the notebook, a [SAIA API key](https://docs.hpc.gwdg.de/services/saia/index.html) is needed. Since the translation process can take some time, it has been outsourced to a separate notebook.

In [245]:
to_translate = prepared_df[["monastery_name", 'Lde', 'Dde', "note"]].copy()
to_translate = to_translate.rename(columns={"Lde": "building_Lde", "Dde": "building_Dde", "monastery_name" : "monastery_Lde", "note": "monastery_Dde"})
to_translate.to_csv("data/translation/to_translate.csv")
to_translate

,monastery_Lde,building_Lde,building_Dde,monastery_Dde
0,Augustinereremitenkloster Windsheim,"""Gebäudekomplex Augustinereremitenkloster Wind...","""Gebäudekomplex Windsheim des Augustinereremit...",NaN
1,"Johanniterkommende Viborg, Dänemark","""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Gebäudekomplex Viborg der Johanniterkommende ...",Lokalisierung nach Ortsmittelpunkt.
2,"Augustinereremitenkloster zum Erlöser, Alsfeld","""Gebäudekomplex Augustinereremitenkloster zum ...","""Gebäudekomplex des Augustinereremitenklosters...",NaN
3,Franziskanerinnenkloster Bondorf,"""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Gebäudekomplex des Franziskanerinnenklosters ...",NaN
4,Augustinerinnenkloster Eldagsen (Marienthal),"""Gebäudekomplex Augustinerinnenkloster Eldagse...","""Gebäudekomplex des Augustinerinnenklosters El...",1647 starb die letzte Konventualin. Koordinate...
5,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...",Seit 1559 zur neugegründeten Diözese Haarlem g...
6,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...",Seit 1559 zur neugegründeten Diözese Haarlem g...
8,Prämonstratenserinnenstift Beselich,"""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Gebäudekomplex des Prämonstratenserinnenstift...",1568 wurde das Kloster in eine Erziehungsansta...
9,"Franziskanerterziarinnenkloster St. Ignatius, ...","""Gebäudekomplex Franziskanerterziarinnenkloste...","""Gebäudekomplex des Franziskanerterziarinnenkl...",Der Beginenkonvent Loershaus in der Stolkgasse...
10,"Franziskanerinnenkloster, gen. Am Sülchen Tor,...","""Gebäudekomplex Franziskanerinnenkloster, gen....","""Gebäudekomplex Rottenburg des Franziskanerinn...",NaN


After executing the above cell, a table is generated in `data/translation` that contains all terms that should be translated: `to_translate.csv`. Execute Notebook 1a. Once the execution is completed, there should be a file named `translated.csv` that contains the translations within the `data/translation` folder. Once the file exists, you can run the next cell to load the translated labels.

In [246]:
translated = pd.read_csv("data/translation/translated.csv")
translated["building_Lde"] = translated["building_Lde"].str.strip().apply(lambda x:f'\"{x}\"' if not pd.isna(x) else np.nan)
prepared_df = pd.merge(prepared_df, translated[["building_Lde", "building_Len"]], how="left", left_on="Lde", right_on="building_Lde").drop_duplicates()
prepared_df.rename(columns={"building_Len":"Len"}, inplace=True)
prepared_df["Len"] = prepared_df["Len"].apply(lambda x:f'\"{x}\"' if not pd.isna(x) else np.nan)
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,location_name,Unnamed: 0_y,id_gsn,status,note,monastery_name,Lde,Dde,building_Lde,Len
0,26,7956,705,60419,1291,NaN,NaN,1525,NaN,NaN,...,Windsheim,2169,60419,Online,NaN,Augustinereremitenkloster Windsheim,"""Gebäudekomplex Augustinereremitenkloster Wind...","""Gebäudekomplex Windsheim des Augustinereremit...","""Gebäudekomplex Augustinereremitenkloster Wind...","""Building complex of the Austin Friars Windshe..."
1,541,17231,46484701,11787,1285,NaN,1285 erstmals erwähnt,1526,NaN,1526 noch existent,...,Viborg,7030,11787,Online,Lokalisierung nach Ortsmittelpunkt.,"Johanniterkommende Viborg, Dänemark","""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Gebäudekomplex Viborg der Johanniterkommende ...","""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Building complex of the Knights Hospitallers ..."
2,924,5344,209,30016,1250,1260.0,um 1255,1527,NaN,NaN,...,NaN,3442,30016,Online,NaN,"Augustinereremitenkloster zum Erlöser, Alsfeld","""Gebäudekomplex Augustinereremitenkloster zum ...","""Gebäudekomplex des Augustinereremitenklosters...","""Gebäudekomplex Augustinereremitenkloster zum ...","""Building complex of the Austin Friars of the ..."
3,982,3064,1420,20004,1357,NaN,NaN,1534,NaN,um 1534,...,NaN,2624,20004,Online,NaN,Franziskanerinnenkloster Bondorf,"""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Gebäudekomplex des Franziskanerinnenklosters ...","""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Building complex of the Franciscan nuns' mona..."
4,1357,68,2803,68,1437,NaN,NaN,1647,NaN,NaN,...,NaN,4227,68,Online,1647 starb die letzte Konventualin. Koordinate...,Augustinerinnenkloster Eldagsen (Marienthal),"""Gebäudekomplex Augustinerinnenkloster Eldagse...","""Gebäudekomplex des Augustinerinnenklosters El...","""Gebäudekomplex Augustinerinnenkloster Eldagse...","""Building complex of the Augustinian nuns' mon..."
5,2440,13102,46483121,8028,1409,NaN,NaN,1415,NaN,NaN,...,Amsterdam,5690,8028,Online,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,..."
7,2441,13103,46483121,8028,1415,NaN,NaN,1579,NaN,NaN,...,Amsterdam,5690,8028,Online,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,..."
9,3283,5599,29638,30279,1163,1185.0,1163/1185,1568,NaN,NaN,...,NaN,1536,30279,Online,1568 wurde das Kloster in eine Erziehungsansta...,Prämonstratenserinnenstift Beselich,"""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Gebäudekomplex des Prämonstratenserinnenstift...","""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Building complex of the Premonstratensian nun..."
10,3819,7133,6305,50211,1245,1295.0,vor 1295,1802,NaN,NaN,...,NaN,1440,50211,Online,Der Beginenkonvent Loershaus in der Stolkgasse...,"Franziskanerterziarinnenkloster St. Ignatius, ...","""Gebäudekomplex Franziskanerterziarinnenkloste...","""Gebäudekomplex des Franziskanerterziarinnenkl...","""Gebäudekomplex Franziskanerterziarinnenkloste...","""Building complex of the Franciscans St. Ignat..."
11,5090,3754,10109,20694,1345,NaN,vor 1345,1493,NaN,NaN,...,Rottenburg,1643,20694,Online,NaN,"Franziskanerinnenkloster, gen. Am Sülchen Tor,...","""Gebäudekomplex Franziskanerinnenkloster, gen....","""Gebäudekomplex Rottenburg des Franziskanerinn...","""Gebäudekomplex Franziskanerinnenkloster, gen....","""Building complex of the Franciscan nuns' mona..."


## Geocoordinates

Our data model separates religious communities from the building complexes in which they lived and worked. The geocoordinates of a location of a religious community are properties of the building complex in this modeling. In the monastery database, there are two levels of accuracy with which the localization of a monastery location can be performed: coordinates for a monastery location will either represent the exact point where the building was located, or the central point of a place, e.g. a village, in which it was located. It is to be noted that the centroid-based location always only represents an approximation of the centroid of the modern location. In cases where the exact location of the building complex is unknown, the respective item will not be linked to any coordinates. Instead, the coordinates of the place where it is located should be queried. In all other cases, the coordinates are directly linked to the building complexes, using values from the `latitude` and `longitude` columns as [P48](https://database.factgrid.de/wiki/Property:P48).

In [247]:
for index, row in prepared_df.iterrows():
    if (not pd.isna(row["latitude"])) and (not pd.isna(row["longitude"])):
        prepared_df.loc[index, "P48"] = f'@{row["latitude"]}/{row["longitude"]}'
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,Unnamed: 0_y,id_gsn,status,note,monastery_name,Lde,Dde,building_Lde,Len,P48
0,26,7956,705,60419,1291,NaN,NaN,1525,NaN,NaN,...,2169,60419,Online,NaN,Augustinereremitenkloster Windsheim,"""Gebäudekomplex Augustinereremitenkloster Wind...","""Gebäudekomplex Windsheim des Augustinereremit...","""Gebäudekomplex Augustinereremitenkloster Wind...","""Building complex of the Austin Friars Windshe...",@49.5032/10.4192
1,541,17231,46484701,11787,1285,NaN,1285 erstmals erwähnt,1526,NaN,1526 noch existent,...,7030,11787,Online,Lokalisierung nach Ortsmittelpunkt.,"Johanniterkommende Viborg, Dänemark","""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Gebäudekomplex Viborg der Johanniterkommende ...","""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Building complex of the Knights Hospitallers ...",NaN
2,924,5344,209,30016,1250,1260.0,um 1255,1527,NaN,NaN,...,3442,30016,Online,NaN,"Augustinereremitenkloster zum Erlöser, Alsfeld","""Gebäudekomplex Augustinereremitenkloster zum ...","""Gebäudekomplex des Augustinereremitenklosters...","""Gebäudekomplex Augustinereremitenkloster zum ...","""Building complex of the Austin Friars of the ...",@50.7496302/9.270021145
3,982,3064,1420,20004,1357,NaN,NaN,1534,NaN,um 1534,...,2624,20004,Online,NaN,Franziskanerinnenkloster Bondorf,"""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Gebäudekomplex des Franziskanerinnenklosters ...","""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Building complex of the Franciscan nuns' mona...",@48.52169237/8.83679565
4,1357,68,2803,68,1437,NaN,NaN,1647,NaN,NaN,...,4227,68,Online,1647 starb die letzte Konventualin. Koordinate...,Augustinerinnenkloster Eldagsen (Marienthal),"""Gebäudekomplex Augustinerinnenkloster Eldagse...","""Gebäudekomplex des Augustinerinnenklosters El...","""Gebäudekomplex Augustinerinnenkloster Eldagse...","""Building complex of the Augustinian nuns' mon...",NaN
5,2440,13102,46483121,8028,1409,NaN,NaN,1415,NaN,NaN,...,5690,8028,Online,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,...",@52.36972222/4.894166667
7,2441,13103,46483121,8028,1415,NaN,NaN,1579,NaN,NaN,...,5690,8028,Online,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,...",@52.3710502700649/4.89711477458891
9,3283,5599,29638,30279,1163,1185.0,1163/1185,1568,NaN,NaN,...,1536,30279,Online,1568 wurde das Kloster in eine Erziehungsansta...,Prämonstratenserinnenstift Beselich,"""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Gebäudekomplex des Prämonstratenserinnenstift...","""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Building complex of the Premonstratensian nun...",@50.44984799/8.140393206
10,3819,7133,6305,50211,1245,1295.0,vor 1295,1802,NaN,NaN,...,1440,50211,Online,Der Beginenkonvent Loershaus in der Stolkgasse...,"Franziskanerterziarinnenkloster St. Ignatius, ...","""Gebäudekomplex Franziskanerterziarinnenkloste...","""Gebäudekomplex des Franziskanerterziarinnenkl...","""Gebäudekomplex Franziskanerterziarinnenkloste...","""Building complex of the Franciscans St. Ignat...",@50.94411182300006/6.953622857186692
11,5090,3754,10109,20694,1345,NaN,vor 1345,1493,NaN,NaN,...,1643,20694,Online,NaN,"Franziskanerinnenkloster, gen. Am Sülchen Tor,...","""Gebäudekomplex Franziskanerinnenkloster, gen....","""Gebäudekomplex Rottenburg des Franziskanerinn...","""Gebäudekomplex Franzi

In [248]:
# Find occurences of identical coordinates
coord_duplicates = prepared_df[prepared_df.duplicated(subset="P48", keep=False)].dropna(subset="P48").drop_duplicates(subset="id_monastery_location", keep=False)
prepared_df.drop_duplicates(subset="P48", inplace=True)
coord_duplicates 

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,Unnamed: 0_y,id_gsn,status,note,monastery_name,Lde,Dde,building_Lde,Len,P48


In [249]:
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,Unnamed: 0_y,id_gsn,status,note,monastery_name,Lde,Dde,building_Lde,Len,P48
0,26,7956,705,60419,1291,NaN,NaN,1525,NaN,NaN,...,2169,60419,Online,NaN,Augustinereremitenkloster Windsheim,"""Gebäudekomplex Augustinereremitenkloster Wind...","""Gebäudekomplex Windsheim des Augustinereremit...","""Gebäudekomplex Augustinereremitenkloster Wind...","""Building complex of the Austin Friars Windshe...",@49.5032/10.4192
1,541,17231,46484701,11787,1285,NaN,1285 erstmals erwähnt,1526,NaN,1526 noch existent,...,7030,11787,Online,Lokalisierung nach Ortsmittelpunkt.,"Johanniterkommende Viborg, Dänemark","""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Gebäudekomplex Viborg der Johanniterkommende ...","""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Building complex of the Knights Hospitallers ...",NaN
2,924,5344,209,30016,1250,1260.0,um 1255,1527,NaN,NaN,...,3442,30016,Online,NaN,"Augustinereremitenkloster zum Erlöser, Alsfeld","""Gebäudekomplex Augustinereremitenkloster zum ...","""Gebäudekomplex des Augustinereremitenklosters...","""Gebäudekomplex Augustinereremitenkloster zum ...","""Building complex of the Austin Friars of the ...",@50.7496302/9.270021145
3,982,3064,1420,20004,1357,NaN,NaN,1534,NaN,um 1534,...,2624,20004,Online,NaN,Franziskanerinnenkloster Bondorf,"""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Gebäudekomplex des Franziskanerinnenklosters ...","""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Building complex of the Franciscan nuns' mona...",@48.52169237/8.83679565
5,2440,13102,46483121,8028,1409,NaN,NaN,1415,NaN,NaN,...,5690,8028,Online,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,...",@52.36972222/4.894166667
7,2441,13103,46483121,8028,1415,NaN,NaN,1579,NaN,NaN,...,5690,8028,Online,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,...",@52.3710502700649/4.89711477458891
9,3283,5599,29638,30279,1163,1185.0,1163/1185,1568,NaN,NaN,...,1536,30279,Online,1568 wurde das Kloster in eine Erziehungsansta...,Prämonstratenserinnenstift Beselich,"""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Gebäudekomplex des Prämonstratenserinnenstift...","""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Building complex of the Premonstratensian nun...",@50.44984799/8.140393206
10,3819,7133,6305,50211,1245,1295.0,vor 1295,1802,NaN,NaN,...,1440,50211,Online,Der Beginenkonvent Loershaus in der Stolkgasse...,"Franziskanerterziarinnenkloster St. Ignatius, ...","""Gebäudekomplex Franziskanerterziarinnenkloste...","""Gebäudekomplex des Franziskanerterziarinnenkl...","""Gebäudekomplex Franziskanerterziarinnenkloste...","""Building complex of the Franciscans St. Ignat...",@50.94411182300006/6.953622857186692
11,5090,3754,10109,20694,1345,NaN,vor 1345,1493,NaN,NaN,...,1643,20694,Online,NaN,"Franziskanerinnenkloster, gen. Am Sülchen Tor,...","""Gebäudekomplex Franziskanerinnenkloster, gen....","""Gebäudekomplex Rottenburg des Franziskanerinn...","""Gebäudekomplex Franziskanerinnenkloster, gen....","""Building complex of the Franciscan nuns' mona...",@48.47682468/8.93405115
13,6124,9838,46479299,5229,1127,NaN,NaN,1525,NaN,NaN,...,3750,5229,Online,"Erst Propstei, seit 1472/1473 Priorat des Wind...",Augustinerchorherrenstift St. Martin auf dem Z...,"""Gebäudekomplex Augustinerchorherrenstift St. ...","""Gebäudekomplex des Augustinerchorher

## Connection to places

The prerequisite for connecting all building complexes with the locations in which they were found is that there are items in FactGrid for these locations. For the collection on locality data in the monastery database, the open source service [geonames](https://www.geonames.org/) was the central tool. Therefore, there is a geonames ID in the monastery database for each location. In FactGrid, there is also a qualifier (P418) for the GeoNames ID. This can be used to assign the location data to each other and to subsequently fill in missing locations. The notebook 1b - Place Matching describes this process.

In order to match all places needed, a matching between FactGrid and the place data from the monastery database is needed. All information that is already available should be placed in a file called `places_reconciled.xlsx` in the `reconciliation` folder. Make sure that the table has at least a column called `place_id` and one called `factgrid_id` that represent the id of the place in the table `gs_places` and in FactGrid respectively. The following cell will load the reconciled places and merge them to the data. If any places remain without a FactGrid id, they will be saved in a new table called `places_without_factgrid.xlsx` in the `reconciliation` folder. Find or create the missing Items in Factgrid and add the information to the `places_reconciled.xlsx` table in the `reconciliation` folder. Afterwards, re-run the workflow. 

In [250]:
# 1. Load the reconciled places
places_reconciled = pd.read_excel("data/reconciliation/places_reconciled.xlsx")[["place_id", "factgrid_id"]]
# 2. Merge them to the table with prepared monasteries
prepared_df = pd.merge(prepared_df, places_reconciled, how="left", on="place_id")
prepared_df = prepared_df.rename(columns={"factgrid_id":"P83"})
prepared_df
# 3. Filter out missing FactGrid Items and store them in a separate table
missing_factgrid_ids = prepared_df[prepared_df['P83'].isna()]
missing_factgrid_ids.to_excel('data/reconciliation/places_without_factGrid.xlsx')
prepared_df = prepared_df.dropna(subset = 'P83')
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,id_gsn,status,note,monastery_name,Lde,Dde,building_Lde,Len,P48,P83
0,26,7956,705,60419,1291,NaN,NaN,1525,NaN,NaN,...,60419,Online,NaN,Augustinereremitenkloster Windsheim,"""Gebäudekomplex Augustinereremitenkloster Wind...","""Gebäudekomplex Windsheim des Augustinereremit...","""Gebäudekomplex Augustinereremitenkloster Wind...","""Building complex of the Austin Friars Windshe...",@49.5032/10.4192,Q82638
1,541,17231,46484701,11787,1285,NaN,1285 erstmals erwähnt,1526,NaN,1526 noch existent,...,11787,Online,Lokalisierung nach Ortsmittelpunkt.,"Johanniterkommende Viborg, Dänemark","""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Gebäudekomplex Viborg der Johanniterkommende ...","""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Building complex of the Knights Hospitallers ...",NaN,Q92914
2,924,5344,209,30016,1250,1260.0,um 1255,1527,NaN,NaN,...,30016,Online,NaN,"Augustinereremitenkloster zum Erlöser, Alsfeld","""Gebäudekomplex Augustinereremitenkloster zum ...","""Gebäudekomplex des Augustinereremitenklosters...","""Gebäudekomplex Augustinereremitenkloster zum ...","""Building complex of the Austin Friars of the ...",@50.7496302/9.270021145,Q82180
3,982,3064,1420,20004,1357,NaN,NaN,1534,NaN,um 1534,...,20004,Online,NaN,Franziskanerinnenkloster Bondorf,"""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Gebäudekomplex des Franziskanerinnenklosters ...","""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Building complex of the Franciscan nuns' mona...",@48.52169237/8.83679565,Q83284
4,2440,13102,46483121,8028,1409,NaN,NaN,1415,NaN,NaN,...,8028,Online,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,...",@52.36972222/4.894166667,Q10300
5,2441,13103,46483121,8028,1415,NaN,NaN,1579,NaN,NaN,...,8028,Online,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,...",@52.3710502700649/4.89711477458891,Q10300
6,3283,5599,29638,30279,1163,1185.0,1163/1185,1568,NaN,NaN,...,30279,Online,1568 wurde das Kloster in eine Erziehungsansta...,Prämonstratenserinnenstift Beselich,"""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Gebäudekomplex des Prämonstratenserinnenstift...","""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Building complex of the Premonstratensian nun...",@50.44984799/8.140393206,Q89418
7,3819,7133,6305,50211,1245,1295.0,vor 1295,1802,NaN,NaN,...,50211,Online,Der Beginenkonvent Loershaus in der Stolkgasse...,"Franziskanerterziarinnenkloster St. Ignatius, ...","""Gebäudekomplex Franziskanerterziarinnenkloste...","""Gebäudekomplex des Franziskanerterziarinnenkl...","""Gebäudekomplex Franziskanerterziarinnenkloste...","""Building complex of the Franciscans St. Ignat...",@50.94411182300006/6.953622857186692,Q10400
8,5090,3754,10109,20694,1345,NaN,vor 1345,1493,NaN,NaN,...,20694,Online,NaN,"Franziskanerinnenkloster, gen. Am Sülchen Tor,...","""Gebäudekomplex Franziskanerinnenkloster, gen....","""Gebäudekomplex Rottenburg des Franziskanerinn...","""Gebäudekomplex Franziskanerinnenkloster, gen....","""Building complex of the Franciscan nuns' mona...",@48.47682468/8.93405115,Q91013
9,6124,9838,46479299,5229,1127,NaN,NaN,1525,NaN,NaN,...,5229,Online,"Erst Propstei, seit 1472/1473 Priorat des Wind...",Augustinerchorherrenstift St. Martin auf dem Z...,"""Gebäudekomplex Augustinerchorherrenstift St. ...","""Gebäudekomplex des Augustinerchorhe

## Instance of statement

To state that these items are building complexes, the Item [Q635758](https://database.factgrid.de/wiki/Item:Q635758) (building complex) is connected to all entries using [P2](https://database.factgrid.de/wiki/Property:P2) (instance of)

In [251]:
prepared_df["P2"] = "Q635758"
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,status,note,monastery_name,Lde,Dde,building_Lde,Len,P48,P83,P2
0,26,7956,705,60419,1291,NaN,NaN,1525,NaN,NaN,...,Online,NaN,Augustinereremitenkloster Windsheim,"""Gebäudekomplex Augustinereremitenkloster Wind...","""Gebäudekomplex Windsheim des Augustinereremit...","""Gebäudekomplex Augustinereremitenkloster Wind...","""Building complex of the Austin Friars Windshe...",@49.5032/10.4192,Q82638,Q635758
1,541,17231,46484701,11787,1285,NaN,1285 erstmals erwähnt,1526,NaN,1526 noch existent,...,Online,Lokalisierung nach Ortsmittelpunkt.,"Johanniterkommende Viborg, Dänemark","""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Gebäudekomplex Viborg der Johanniterkommende ...","""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Building complex of the Knights Hospitallers ...",NaN,Q92914,Q635758
2,924,5344,209,30016,1250,1260.0,um 1255,1527,NaN,NaN,...,Online,NaN,"Augustinereremitenkloster zum Erlöser, Alsfeld","""Gebäudekomplex Augustinereremitenkloster zum ...","""Gebäudekomplex des Augustinereremitenklosters...","""Gebäudekomplex Augustinereremitenkloster zum ...","""Building complex of the Austin Friars of the ...",@50.7496302/9.270021145,Q82180,Q635758
3,982,3064,1420,20004,1357,NaN,NaN,1534,NaN,um 1534,...,Online,NaN,Franziskanerinnenkloster Bondorf,"""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Gebäudekomplex des Franziskanerinnenklosters ...","""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Building complex of the Franciscan nuns' mona...",@48.52169237/8.83679565,Q83284,Q635758
4,2440,13102,46483121,8028,1409,NaN,NaN,1415,NaN,NaN,...,Online,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,...",@52.36972222/4.894166667,Q10300,Q635758
5,2441,13103,46483121,8028,1415,NaN,NaN,1579,NaN,NaN,...,Online,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,...",@52.3710502700649/4.89711477458891,Q10300,Q635758
6,3283,5599,29638,30279,1163,1185.0,1163/1185,1568,NaN,NaN,...,Online,1568 wurde das Kloster in eine Erziehungsansta...,Prämonstratenserinnenstift Beselich,"""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Gebäudekomplex des Prämonstratenserinnenstift...","""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Building complex of the Premonstratensian nun...",@50.44984799/8.140393206,Q89418,Q635758
7,3819,7133,6305,50211,1245,1295.0,vor 1295,1802,NaN,NaN,...,Online,Der Beginenkonvent Loershaus in der Stolkgasse...,"Franziskanerterziarinnenkloster St. Ignatius, ...","""Gebäudekomplex Franziskanerterziarinnenkloste...","""Gebäudekomplex des Franziskanerterziarinnenkl...","""Gebäudekomplex Franziskanerterziarinnenkloste...","""Building complex of the Franciscans St. Ignat...",@50.94411182300006/6.953622857186692,Q10400,Q635758
8,5090,3754,10109,20694,1345,NaN,vor 1345,1493,NaN,NaN,...,Online,NaN,"Franziskanerinnenkloster, gen. Am Sülchen Tor,...","""Gebäudekomplex Franziskanerinnenkloster, gen....","""Gebäudekomplex Rottenburg des Franziskanerinn...","""Gebäudekomplex Franziskanerinnenkloster, gen....","""Building complex of the Franciscan nuns' mona...",@48.47682468/8.93405115,Q91013,Q635758
9,6124,9838,46479299,5229,1127,NaN,NaN,1525,NaN,NaN,...,Online,"Erst Propstei, seit 1472/1473 Priorat des Wind...",Augustinerchorherrenstift St. Martin auf dem Z...,"""Gebäudekomplex Augustinerchorherrenstift St. ...","""Gebäudekomplex des Augus

## Vocabulary Terms

In order to keep a mapping between the monastery database and FactGrid, every item will receive a distinct vocabulary term that is constructed using the `id_monastery_location` from the `gs_monastery_location` table. The FactGrid Property to use is [P1301](https://database.factgrid.de/wiki/Property:P1301) (GS vocabulary term). For the construction, the following pattern is being used: `GSMonasteryLocation<id_monastery_location>`.

In [252]:
prepared_df['P1301'] = prepared_df['id_monastery_location'].apply(lambda x: f'\"GSMonasteryLocation{x}\"')
# Handle Vocabulary Terms for duplicated coords
for index, row in prepared_df.iterrows():
    if row["id_monastery_location"] in coord_duplicates["id_monastery_location"].values:
        x = 0
        for index, row in coord_duplicates[coord_duplicates["gsn_id"] == row["gsn_id"]].iterrows():
            if x == 0:
                x += 1
                continue
            else:
                prepared_df.loc[prepared_df["gsn_id"] == row["gsn_id"], f"P1301.{x}"] = f'\"GSMonasteryLocation{row["id_monastery_location"]}\"'
                x += 1
            
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,note,monastery_name,Lde,Dde,building_Lde,Len,P48,P83,P2,P1301
0,26,7956,705,60419,1291,NaN,NaN,1525,NaN,NaN,...,NaN,Augustinereremitenkloster Windsheim,"""Gebäudekomplex Augustinereremitenkloster Wind...","""Gebäudekomplex Windsheim des Augustinereremit...","""Gebäudekomplex Augustinereremitenkloster Wind...","""Building complex of the Austin Friars Windshe...",@49.5032/10.4192,Q82638,Q635758,"""GSMonasteryLocation7956"""
1,541,17231,46484701,11787,1285,NaN,1285 erstmals erwähnt,1526,NaN,1526 noch existent,...,Lokalisierung nach Ortsmittelpunkt.,"Johanniterkommende Viborg, Dänemark","""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Gebäudekomplex Viborg der Johanniterkommende ...","""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Building complex of the Knights Hospitallers ...",NaN,Q92914,Q635758,"""GSMonasteryLocation17231"""
2,924,5344,209,30016,1250,1260.0,um 1255,1527,NaN,NaN,...,NaN,"Augustinereremitenkloster zum Erlöser, Alsfeld","""Gebäudekomplex Augustinereremitenkloster zum ...","""Gebäudekomplex des Augustinereremitenklosters...","""Gebäudekomplex Augustinereremitenkloster zum ...","""Building complex of the Austin Friars of the ...",@50.7496302/9.270021145,Q82180,Q635758,"""GSMonasteryLocation5344"""
3,982,3064,1420,20004,1357,NaN,NaN,1534,NaN,um 1534,...,NaN,Franziskanerinnenkloster Bondorf,"""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Gebäudekomplex des Franziskanerinnenklosters ...","""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Building complex of the Franciscan nuns' mona...",@48.52169237/8.83679565,Q83284,Q635758,"""GSMonasteryLocation3064"""
4,2440,13102,46483121,8028,1409,NaN,NaN,1415,NaN,NaN,...,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,...",@52.36972222/4.894166667,Q10300,Q635758,"""GSMonasteryLocation13102"""
5,2441,13103,46483121,8028,1415,NaN,NaN,1579,NaN,NaN,...,Seit 1559 zur neugegründeten Diözese Haarlem g...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,...",@52.3710502700649/4.89711477458891,Q10300,Q635758,"""GSMonasteryLocation13103"""
6,3283,5599,29638,30279,1163,1185.0,1163/1185,1568,NaN,NaN,...,1568 wurde das Kloster in eine Erziehungsansta...,Prämonstratenserinnenstift Beselich,"""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Gebäudekomplex des Prämonstratenserinnenstift...","""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Building complex of the Premonstratensian nun...",@50.44984799/8.140393206,Q89418,Q635758,"""GSMonasteryLocation5599"""
7,3819,7133,6305,50211,1245,1295.0,vor 1295,1802,NaN,NaN,...,Der Beginenkonvent Loershaus in der Stolkgasse...,"Franziskanerterziarinnenkloster St. Ignatius, ...","""Gebäudekomplex Franziskanerterziarinnenkloste...","""Gebäudekomplex des Franziskanerterziarinnenkl...","""Gebäudekomplex Franziskanerterziarinnenkloste...","""Building complex of the Franciscans St. Ignat...",@50.94411182300006/6.953622857186692,Q10400,Q635758,"""GSMonasteryLocation7133"""
8,5090,3754,10109,20694,1345,NaN,vor 1345,1493,NaN,NaN,...,NaN,"Franziskanerinnenkloster, gen. Am Sülchen Tor,...","""Gebäudekomplex Franziskanerinnenkloster, gen....","""Gebäudekomplex Rottenburg des Franziskanerinn...","""Gebäudekomplex Franziskanerinnenkloster, gen....","""Building complex of the Franciscan nuns' mona...",@48.47682468/8.93405115,Q91013,Q635758,"""GSMonasteryLocation3754"""
9,6124,9838,46479299,5229,1127,NaN,N

In [253]:
building_complexes_with_coordinates = prepared_df[["gsn_id", "P1301", "P83", "P48", "location_begin_tpq", "location_begin_taq", "location_end_tpq", "location_end_taq", "place_id"]].rename(columns={"P1301":"id_monastery_location", "P83":"place_factgrid", "P48":"coordinates"})
building_complexes_with_coordinates["id_monastery_location"] = building_complexes_with_coordinates["id_monastery_location"].str.strip("\"").str.split("Location").str[-1].astype(int)
building_complexes_with_coordinates["latitude"] = building_complexes_with_coordinates["coordinates"].str.split("/").str[0].str[1:].astype(float)
building_complexes_with_coordinates["longitude"] = building_complexes_with_coordinates["coordinates"].str.split("/").str[1].astype(float)
building_complexes_with_coordinates.drop(columns=["coordinates"])
building_complexes_with_coordinates["place_id"] = building_complexes_with_coordinates["place_id"].astype(int)
building_complexes_with_coordinates.to_csv("data/intermediate_results/building_complexes_coordinates.csv")
building_complexes_with_coordinates

,gsn_id,id_monastery_location,place_factgrid,coordinates,location_begin_tpq,location_begin_taq,location_end_tpq,location_end_taq,place_id,latitude,longitude
0,60419,7956,Q82638,@49.5032/10.4192,1291,NaN,1525,NaN,705,49.503200,10.419200
1,11787,17231,Q92914,NaN,1285,NaN,1526,NaN,46484701,NaN,NaN
2,30016,5344,Q82180,@50.7496302/9.270021145,1250,1260.0,1527,NaN,209,50.749630,9.270021
3,20004,3064,Q83284,@48.52169237/8.83679565,1357,NaN,1534,NaN,1420,48.521692,8.836796
4,8028,13102,Q10300,@52.36972222/4.894166667,1409,NaN,1415,NaN,46483121,52.369722,4.894167
5,8028,13103,Q10300,@52.3710502700649/4.89711477458891,1415,NaN,1579,NaN,46483121,52.371050,4.897115
6,30279,5599,Q89418,@50.44984799/8.140393206,1163,1185.0,1568,NaN,29638,50.449848,8.140393
7,50211,7133,Q10400,@50.94411182300006/6.953622857186692,1245,1295.0,1802,NaN,6305,50.944112,6.953623
8,20694,3754,Q91013,@48.47682468/8.93405115,1345,NaN,1493,NaN,10109,48.476825,8.934051
9,5229,9838,Q10505,@47.387139/8.57533,1127,NaN,1525,NaN,46479299,47.387139,8.575330


## Dioceses

By connecting to modern municipalities, it is possible to understand in which territorial structures the (former) building complexes are located today. However, the monastery database also contains information about the historical diocese in which the building complexes were located. This information is stored in the table `gs_places` in the column `diocese_id`. Therefore, the locations where monastery locations are located are assigned to a diocese. In FactGrid, we connect the information about the dioceses directly to the building complexes. A building complex has a property [P1003](https://database.factgrid.de/wiki/Item:Q21662) (Diocese), which connects to a diocese item, for example the Archdiocese of Mainz ([Q153230](https://database.factgrid.de/wiki/Item:Q153230)). The historical affiliation of a location to a diocese is a complex phenomenon. On the one hand, this changed over time, especially in border areas. On the other hand, it is also possible that an area that we understand today as a contiguous location was not a contiguous location around 1500 and only partially belonged to a certain diocese. Therefore, we separate the modern territorial localization (statements about the current location of the address) from the historical localization (statements about the affiliation to a diocese).

In [254]:
# Merge gs_places['diocese_id] to existing table
places_selection = dataframes["gs_places"][["id_places", "diocese_id"]]
diocese_urls_selection = dataframes["gs_id_external_urls_diocese"][dataframes["gs_id_external_urls_diocese"]["url_type_id"]==42][["diocese_id", "url_value"]]
diocese_urls_selection
prepared_df = pd.merge(prepared_df, places_selection, how="left", left_on="place_id", right_on="id_places").drop(columns="id_places")
prepared_df = pd.merge(prepared_df, diocese_urls_selection, how="left", left_on="diocese_id", right_on="diocese_id").drop(columns="diocese_id").rename(columns={"url_value":"P1003"})

# Handle dioceses for coordinate duplicates
coord_duplicates = pd.merge(coord_duplicates, places_selection, how="left", left_on="place_id", right_on="id_places").drop(columns="id_places")
coord_duplicates = pd.merge(coord_duplicates, diocese_urls_selection, how="left", left_on="diocese_id", right_on="diocese_id").drop(columns="diocese_id").rename(columns={"url_value":"P1003"})
for index, row in prepared_df.iterrows():
    if row["id_monastery_location"] in coord_duplicates["id_monastery_location"].values:
        x = 0
        for index, row in coord_duplicates[coord_duplicates["gsn_id"] == row["gsn_id"]].iterrows():
            if x == 0:
                x += 1
                continue
            else:
                prepared_df.loc[prepared_df["gsn_id"] == row["gsn_id"], f"P1003.{x}"] = row["P1003"]
                x += 1
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,monastery_name,Lde,Dde,building_Lde,Len,P48,P83,P2,P1301,P1003
0,26,7956,705,60419,1291,NaN,NaN,1525,NaN,NaN,...,Augustinereremitenkloster Windsheim,"""Gebäudekomplex Augustinereremitenkloster Wind...","""Gebäudekomplex Windsheim des Augustinereremit...","""Gebäudekomplex Augustinereremitenkloster Wind...","""Building complex of the Austin Friars Windshe...",@49.5032/10.4192,Q82638,Q635758,"""GSMonasteryLocation7956""",Q153247
1,541,17231,46484701,11787,1285,NaN,1285 erstmals erwähnt,1526,NaN,1526 noch existent,...,"Johanniterkommende Viborg, Dänemark","""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Gebäudekomplex Viborg der Johanniterkommende ...","""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Building complex of the Knights Hospitallers ...",NaN,Q92914,Q635758,"""GSMonasteryLocation17231""",Q153625
2,924,5344,209,30016,1250,1260.0,um 1255,1527,NaN,NaN,...,"Augustinereremitenkloster zum Erlöser, Alsfeld","""Gebäudekomplex Augustinereremitenkloster zum ...","""Gebäudekomplex des Augustinereremitenklosters...","""Gebäudekomplex Augustinereremitenkloster zum ...","""Building complex of the Austin Friars of the ...",@50.7496302/9.270021145,Q82180,Q635758,"""GSMonasteryLocation5344""",Q153230
3,982,3064,1420,20004,1357,NaN,NaN,1534,NaN,um 1534,...,Franziskanerinnenkloster Bondorf,"""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Gebäudekomplex des Franziskanerinnenklosters ...","""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Building complex of the Franciscan nuns' mona...",@48.52169237/8.83679565,Q83284,Q635758,"""GSMonasteryLocation3064""",Q153226
4,2440,13102,46483121,8028,1409,NaN,NaN,1415,NaN,NaN,...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,...",@52.36972222/4.894166667,Q10300,Q635758,"""GSMonasteryLocation13102""",Q153266
5,2441,13103,46483121,8028,1415,NaN,NaN,1579,NaN,NaN,...,"Franziskanertertiarenkloster St. Paul, Amsterd...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,...",@52.3710502700649/4.89711477458891,Q10300,Q635758,"""GSMonasteryLocation13103""",Q153266
6,3283,5599,29638,30279,1163,1185.0,1163/1185,1568,NaN,NaN,...,Prämonstratenserinnenstift Beselich,"""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Gebäudekomplex des Prämonstratenserinnenstift...","""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Building complex of the Premonstratensian nun...",@50.44984799/8.140393206,Q89418,Q635758,"""GSMonasteryLocation5599""",Q153244
7,3819,7133,6305,50211,1245,1295.0,vor 1295,1802,NaN,NaN,...,"Franziskanerterziarinnenkloster St. Ignatius, ...","""Gebäudekomplex Franziskanerterziarinnenkloste...","""Gebäudekomplex des Franziskanerterziarinnenkl...","""Gebäudekomplex Franziskanerterziarinnenkloste...","""Building complex of the Franciscans St. Ignat...",@50.94411182300006/6.953622857186692,Q10400,Q635758,"""GSMonasteryLocation7133""",Q153225
8,5090,3754,10109,20694,1345,NaN,vor 1345,1493,NaN,NaN,...,"Franziskanerinnenkloster, gen. Am Sülchen Tor,...","""Gebäudekomplex Franziskanerinnenkloster, gen....","""Gebäudekomplex Rottenburg des Franziskanerinn...","""Gebäudekomplex Franziskanerinnenkloster, gen....","""Building complex of the Franciscan nuns' mona...",@48.47682468/8.93405115,Q91013,Q635758,"""GSMonasteryLocation3754""",Q153226
9,6124,9838,46479299,5229,1127,NaN,NaN,1525,NaN,NaN,...,Augustinerchorherrenstift St. Martin auf dem Z...,"""Gebäudekomplex Augustinerchorherrenstift St. ...","""Gebäudekomplex des Augustinerchorherrenstifts...","""

## External Identifiers
In some instances, the monastery database has listed wikipedia articles that are specifically written about the building complex of a monastery. Where these exist, they should be linked to the building complex item. 

In [255]:
gs_external_url_type_with_factgrid = dataframes["gs_external_url_type_with_factgrid"].dropna(subset="factgrid_property")
url_factgrid = pd.merge(dataframes["gs_external_urls_monastery"], gs_external_url_type_with_factgrid, how="left", left_on="url_type_id", right_on="id_url_type")[["gsn_id", "url_value", "factgrid_property", "url_name_formatter"]].dropna(subset="factgrid_property")
for index, row in url_factgrid.iterrows():
    if row["gsn_id"] in prepared_df["id_gsn"].values and "Wikipedia-Artikel zum Baudenkmal" in row["url_name_formatter"]:
        prepared_df.loc[prepared_df["id_gsn"] == row["gsn_id"], row["factgrid_property"]] = f'\"{row["url_value"]}\"'
prepared_df

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,Lde,Dde,building_Lde,Len,P48,P83,P2,P1301,P1003,Sdewiki
0,26,7956,705,60419,1291,NaN,NaN,1525,NaN,NaN,...,"""Gebäudekomplex Augustinereremitenkloster Wind...","""Gebäudekomplex Windsheim des Augustinereremit...","""Gebäudekomplex Augustinereremitenkloster Wind...","""Building complex of the Austin Friars Windshe...",@49.5032/10.4192,Q82638,Q635758,"""GSMonasteryLocation7956""",Q153247,NaN
1,541,17231,46484701,11787,1285,NaN,1285 erstmals erwähnt,1526,NaN,1526 noch existent,...,"""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Gebäudekomplex Viborg der Johanniterkommende ...","""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Building complex of the Knights Hospitallers ...",NaN,Q92914,Q635758,"""GSMonasteryLocation17231""",Q153625,NaN
2,924,5344,209,30016,1250,1260.0,um 1255,1527,NaN,NaN,...,"""Gebäudekomplex Augustinereremitenkloster zum ...","""Gebäudekomplex des Augustinereremitenklosters...","""Gebäudekomplex Augustinereremitenkloster zum ...","""Building complex of the Austin Friars of the ...",@50.7496302/9.270021145,Q82180,Q635758,"""GSMonasteryLocation5344""",Q153230,NaN
3,982,3064,1420,20004,1357,NaN,NaN,1534,NaN,um 1534,...,"""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Gebäudekomplex des Franziskanerinnenklosters ...","""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Building complex of the Franciscan nuns' mona...",@48.52169237/8.83679565,Q83284,Q635758,"""GSMonasteryLocation3064""",Q153226,NaN
4,2440,13102,46483121,8028,1409,NaN,NaN,1415,NaN,NaN,...,"""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,...",@52.36972222/4.894166667,Q10300,Q635758,"""GSMonasteryLocation13102""",Q153266,NaN
5,2441,13103,46483121,8028,1415,NaN,NaN,1579,NaN,NaN,...,"""Gebäudekomplex Franziskanertertiarenkloster S...","""Gebäudekomplex Amsterdam des Franziskanertert...","""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,...",@52.3710502700649/4.89711477458891,Q10300,Q635758,"""GSMonasteryLocation13103""",Q153266,NaN
6,3283,5599,29638,30279,1163,1185.0,1163/1185,1568,NaN,NaN,...,"""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Gebäudekomplex des Prämonstratenserinnenstift...","""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Building complex of the Premonstratensian nun...",@50.44984799/8.140393206,Q89418,Q635758,"""GSMonasteryLocation5599""",Q153244,NaN
7,3819,7133,6305,50211,1245,1295.0,vor 1295,1802,NaN,NaN,...,"""Gebäudekomplex Franziskanerterziarinnenkloste...","""Gebäudekomplex des Franziskanerterziarinnenkl...","""Gebäudekomplex Franziskanerterziarinnenkloste...","""Building complex of the Franciscans St. Ignat...",@50.94411182300006/6.953622857186692,Q10400,Q635758,"""GSMonasteryLocation7133""",Q153225,NaN
8,5090,3754,10109,20694,1345,NaN,vor 1345,1493,NaN,NaN,...,"""Gebäudekomplex Franziskanerinnenkloster, gen....","""Gebäudekomplex Rottenburg des Franziskanerinn...","""Gebäudekomplex Franziskanerinnenkloster, gen....","""Building complex of the Franciscan nuns' mona...",@48.47682468/8.93405115,Q91013,Q635758,"""GSMonasteryLocation3754""",Q153226,NaN
9,6124,9838,46479299,5229,1127,NaN,NaN,1525,NaN,NaN,...,"""Gebäudekomplex Augustinerchorherrenstift St. ...","""Gebäudekomplex des Augustinerchorherrenstifts...","""Gebäudekomplex Augustinerchorherrenstift St. ...","""Building complex of the Canons Regular of St ...",@47.387139/8.57533,Q10505,Q635758,"""GSMonasteryLocation9838""",Q153226,NaN


## Sources / References

Every Statement in FactGrid should be supported by a Source/Reference. To achieve this, a source column `S471` is added after each relevant property to link to the Monastery Database Entries using the Property [P471](https://database.factgrid.de/wiki/Property:P471).

In [256]:
final_table = prepared_df.copy()
for colname in ["P48", "P83", "P1003"]:
    final_table.insert(final_table.columns.get_loc(colname)+1, "S471", final_table["gsn_id"].apply(lambda x:f'\"{x}\"'), allow_duplicates=True)
final_table["P131"] = "Q153178"
final_table

,Unnamed: 0_x,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,...,P48,S471,P83,S471,P2,P1301,P1003,S471,Sdewiki,P131
0,26,7956,705,60419,1291,NaN,NaN,1525,NaN,NaN,...,@49.5032/10.4192,"""60419""",Q82638,"""60419""",Q635758,"""GSMonasteryLocation7956""",Q153247,"""60419""",NaN,Q153178
1,541,17231,46484701,11787,1285,NaN,1285 erstmals erwähnt,1526,NaN,1526 noch existent,...,NaN,"""11787""",Q92914,"""11787""",Q635758,"""GSMonasteryLocation17231""",Q153625,"""11787""",NaN,Q153178
2,924,5344,209,30016,1250,1260.0,um 1255,1527,NaN,NaN,...,@50.7496302/9.270021145,"""30016""",Q82180,"""30016""",Q635758,"""GSMonasteryLocation5344""",Q153230,"""30016""",NaN,Q153178
3,982,3064,1420,20004,1357,NaN,NaN,1534,NaN,um 1534,...,@48.52169237/8.83679565,"""20004""",Q83284,"""20004""",Q635758,"""GSMonasteryLocation3064""",Q153226,"""20004""",NaN,Q153178
4,2440,13102,46483121,8028,1409,NaN,NaN,1415,NaN,NaN,...,@52.36972222/4.894166667,"""8028""",Q10300,"""8028""",Q635758,"""GSMonasteryLocation13102""",Q153266,"""8028""",NaN,Q153178
5,2441,13103,46483121,8028,1415,NaN,NaN,1579,NaN,NaN,...,@52.3710502700649/4.89711477458891,"""8028""",Q10300,"""8028""",Q635758,"""GSMonasteryLocation13103""",Q153266,"""8028""",NaN,Q153178
6,3283,5599,29638,30279,1163,1185.0,1163/1185,1568,NaN,NaN,...,@50.44984799/8.140393206,"""30279""",Q89418,"""30279""",Q635758,"""GSMonasteryLocation5599""",Q153244,"""30279""",NaN,Q153178
7,3819,7133,6305,50211,1245,1295.0,vor 1295,1802,NaN,NaN,...,@50.94411182300006/6.953622857186692,"""50211""",Q10400,"""50211""",Q635758,"""GSMonasteryLocation7133""",Q153225,"""50211""",NaN,Q153178
8,5090,3754,10109,20694,1345,NaN,vor 1345,1493,NaN,NaN,...,@48.47682468/8.93405115,"""20694""",Q91013,"""20694""",Q635758,"""GSMonasteryLocation3754""",Q153226,"""20694""",NaN,Q153178
9,6124,9838,46479299,5229,1127,NaN,NaN,1525,NaN,NaN,...,@47.387139/8.57533,"""5229""",Q10505,"""5229""",Q635758,"""GSMonasteryLocation9838""",Q153226,"""5229""",NaN,Q153178


## Finalizing

To finalize, the table is cleaned up and transformed into a variety of formats. Most importantly, you will find the V1-statements to create the new building complex items under `data/results/building_complexes/import_building_complexes.tsv`

In [257]:
from helper_functions import df_to_qs_v1

final_table["id_monastery_location"].to_csv("data/intermediate_results/new_building_complex_locations_ids.csv")

final_table = final_table.drop(columns=["Dde", "Unnamed: 0_x", "Unnamed: 0_y", "note", "building_Lde", "id_monastery_location", "place_id", "gsn_id", "location_begin_tpq", "location_begin_taq", "location_begin_note", "location_end_tpq", "location_end_taq", "location_end_note", "longitude", "latitude", "location_name", "id_gsn", "status", "monastery_name"])
final_table.insert(0, "qid", np.nan)
final_table.to_excel("data/results/building_complexes/import_building_complexes.xlsx", index=False)
final_table.to_csv("data/results/building_complexes/import_building_complexes.csv", index=False, doublequote=False, quoting=csv.QUOTE_NONE, escapechar="§") #hack to save in Quickstatements-applicable format
with open("data/results/building_complexes/import_building_complexes.csv", "r") as file:
    s = file.read()
with open("data/results/building_complexes/import_building_complexes.csv", "w") as file:
    file.write(s.replace("§", ""))
with open("data/results/building_complexes/import_building_complexes.tsv", "w") as file:
    file.write(df_to_qs_v1(final_table))


final_table

,qid,Lde,Len,P48,S471,P83,S471.1,P2,P1301,P1003,S471.2,Sdewiki,P131
0,NaN,"""Gebäudekomplex Augustinereremitenkloster Wind...","""Building complex of the Austin Friars Windshe...",@49.5032/10.4192,"""60419""",Q82638,"""60419""",Q635758,"""GSMonasteryLocation7956""",Q153247,"""60419""",NaN,Q153178
1,NaN,"""Gebäudekomplex Johanniterkommende Viborg, Dän...","""Building complex of the Knights Hospitallers ...",NaN,"""11787""",Q92914,"""11787""",Q635758,"""GSMonasteryLocation17231""",Q153625,"""11787""",NaN,Q153178
2,NaN,"""Gebäudekomplex Augustinereremitenkloster zum ...","""Building complex of the Austin Friars of the ...",@50.7496302/9.270021145,"""30016""",Q82180,"""30016""",Q635758,"""GSMonasteryLocation5344""",Q153230,"""30016""",NaN,Q153178
3,NaN,"""Gebäudekomplex Franziskanerinnenkloster Bondorf""","""Building complex of the Franciscan nuns' mona...",@48.52169237/8.83679565,"""20004""",Q83284,"""20004""",Q635758,"""GSMonasteryLocation3064""",Q153226,"""20004""",NaN,Q153178
4,NaN,"""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,...",@52.36972222/4.894166667,"""8028""",Q10300,"""8028""",Q635758,"""GSMonasteryLocation13102""",Q153266,"""8028""",NaN,Q153178
5,NaN,"""Gebäudekomplex Franziskanertertiarenkloster S...","""Building complex of the Franciscans St. Paul,...",@52.3710502700649/4.89711477458891,"""8028""",Q10300,"""8028""",Q635758,"""GSMonasteryLocation13103""",Q153266,"""8028""",NaN,Q153178
6,NaN,"""Gebäudekomplex Prämonstratenserinnenstift Bes...","""Building complex of the Premonstratensian nun...",@50.44984799/8.140393206,"""30279""",Q89418,"""30279""",Q635758,"""GSMonasteryLocation5599""",Q153244,"""30279""",NaN,Q153178
7,NaN,"""Gebäudekomplex Franziskanerterziarinnenkloste...","""Building complex of the Franciscans St. Ignat...",@50.94411182300006/6.953622857186692,"""50211""",Q10400,"""50211""",Q635758,"""GSMonasteryLocation7133""",Q153225,"""50211""",NaN,Q153178
8,NaN,"""Gebäudekomplex Franziskanerinnenkloster, gen....","""Building complex of the Franciscan nuns' mona...",@48.47682468/8.93405115,"""20694""",Q91013,"""20694""",Q635758,"""GSMonasteryLocation3754""",Q153226,"""20694""",NaN,Q153178
9,NaN,"""Gebäudekomplex Augustinerchorherrenstift St. ...","""Building complex of the Canons Regular of St ...",@47.387139/8.57533,"""5229""",Q10505,"""5229""",Q635758,"""GSMonasteryLocation9838""",Q153226,"""5229""",NaN,Q153178


## Next steps
As a next step, you should run notebook 2 - Monasteries to create the religious community items that go together with the building complexes. Afterwards you can copy the V1 statements from both, `data/results/building_complexes/import_building_complexes.csv` and `data/results/monasteries/import_monasteries.csv` to Quickstatements and upload.